# Importazione librerie

In [2]:
import numpy as np
import tensorflow as tf
import optuna
import os

from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix

from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.models import Model

import gc

## Load Data

In [3]:
# Directory containing images
whistleImagesDirectory = 'Data\\Sobel\\Whistle'
noiseImagesDirectory = 'Data\\Sobel\\Noise'

# List all image files in the directory
whistleFiles = os.listdir(whistleImagesDirectory)
noiseFiles = os.listdir(noiseImagesDirectory)

# Initialize empty lists for images and labels
images = []
labels = []

# Load and preprocess whistle images
for image_file in whistleFiles:
    image_path = os.path.join(whistleImagesDirectory, image_file)
    image = Image.open(image_path).convert('L')  # Convert to grayscale
    image = image.resize((224, 224))  # Resize
    image = np.array(image)  # Convert to NumPy array
    images.append(image)
    labels.append(1)    #etichetta delfino


# Load and preprocess noise images
for image_file in noiseFiles:
    image_path = os.path.join(noiseImagesDirectory, image_file)
    image = Image.open(image_path).convert('L')  # Convert to grayscale
    image = image.resize((224, 224))
    image = np.array(image)  # Convert to NumPy array
    images.append(image)
    labels.append(0)    #etichetta rumore

# Convert lists to NumPy arrays
X = np.array(images)/255.0
y = np.array(labels)

# Split the dataset into training and test sets                 (90/10)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_val, y_val = [], []

# Split the dataset into training, validation, and test sets    (70/20/10)
#X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
#X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)

# Print the shapes of the resulting sets
print("Train set shapes:", X_train.shape, y_train.shape)
#print("Validation set shapes:", X_val.shape, y_val.shape)
print("Test set shapes:", X_test.shape, y_test.shape)

Train set shapes: (1367, 224, 224) (1367,)
Test set shapes: (152, 224, 224) (152,)


# Optuna
Utilizzo di Optuna per la scelta dell’architettura della rete neurale, ovvero un framework per l'ottimizzazione degli iperparametri. Sceglie automaticamente gli iperparametri di un modello e lo allena, poi valuta la rete migliore basandosi sul registro storico dei tentativi. Il numero di tentativi (_trial_) minimo consigliato è 50, ma per casi più importanti può essere impostato anche a centinaia.
In questo modo, viene generata una rete che ha da 3 a 5 Conv2D layers, ognuno con funzione di attivazione relu, kernel size (3,3), numero di filtri uno tra [16, 32, 64, 128, 256, 512], seguito da un MaxPooling2D.
Viene utilizzato un flatten e infine vengono inseriti uno o due Dense layer con un numero di neuroni tra [64, 128, 256, 512, 1024], più un Dense layer finale.


In [ ]:
def create_model(trial):
    input = Input(shape=(224, 224, 1))

    # Crea layer convoluzionali
    x = input
    for i in range(trial.suggest_int("n_layers", 3, 5)):
        filters=trial.suggest_categorical(f"filters_{i}", [16, 32, 64, 128, 256, 512])
        x = Conv2D(
            filters=filters,
            kernel_size=(3, 3),
            activation="relu"
        )(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        print(filters)

    x = Flatten()(x)

    # Crea dense layers
    for i in range(trial.suggest_int("n_dense_layers", 1, 2)):
        units = trial.suggest_categorical(f"dense_units_{i}", [64, 128, 256, 512, 1024])
        x = Dense(
            units=units,
            activation="relu"
        )(x)
        print('Dense', units)

    # Layer di output
    output = Dense(1, activation="sigmoid")(x)
    print('Dense', 1)

    model = Model(inputs=input, outputs=output)
    return model

def objective(trial):
    model = create_model(trial)

    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

    early_stop = EarlyStopping(monitor='val_loss', patience=15, verbose=1)
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=120, batch_size=32, callbacks=[early_stop])

    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred > 0.5).astype(np.int32)  # Converte probabilità in valori binari

    # Usa l'accuratezza di validazione come metrica da ottimizzare
    precision = precision_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)

    print(f'Precision: {precision}')
    print(f'Recall: {recall}')

    return precision + precision + recall


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Numero di trials: ", len(study.trials))
print("Miglior trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))